In [1]:
# Start final project import modules
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
import sys
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder 
from sklearn import feature_extraction
from __future__ import print_function
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [2]:
# read in the datasets
 
jd = pd.read_csv('job_data_dict_30_sample.csv')


In [3]:
#1. Explore the dataset: 
    # There is a mix of string and numeric data. There are some variables that can be converted to dummy or binary values.
    # While there are no NAs there are unknown values in some of the string data which may need to be treated as NAs.

#jd.describe(include='all')
jd.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5310 entries, 0 to 5309
Data columns (total 5 columns):
Unnamed: 0    5310 non-null int64
jobs          5310 non-null object
jobtitle      5310 non-null object
dictionary    5310 non-null object
id            5310 non-null int64
dtypes: int64(2), object(3)
memory usage: 248.9+ KB


In [4]:
jd.head()

,Unnamed: 0,jobs,jobtitle,dictionary,id
0,53,public relations specialists,event coordinator sales marketing account mana...,event coordinator sales marketing account mana...,49
1,61,sales reps technical and scientific products,account coordinator it,account coordinator it account coordinator it ...,57
2,78,public relations specialists,account project coordinator,account project coordinator account project co...,74
3,91,advertising and promotions managers,account director advertising agency,account director advertising agency account di...,86
4,104,advertising and promotions managers,sr account director,sr account director sr account director classi...,99


In [5]:

cat_columns = ['jobs','jobtitle','dictionary']
feature_columns = ['jobtitle','dictionary']   
string_variables = ['jobs']

jd_2 = jd[cat_columns]
jd_2.head()

,jobs,jobtitle,dictionary
0,public relations specialists,event coordinator sales marketing account mana...,event coordinator sales marketing account mana...
1,sales reps technical and scientific products,account coordinator it,account coordinator it account coordinator it ...
2,public relations specialists,account project coordinator,account project coordinator account project co...
3,advertising and promotions managers,account director advertising agency,account director advertising agency account di...
4,advertising and promotions managers,sr account director,sr account director sr account director classi...


In [6]:
jd_2.jobs.value_counts().min()

30

In [7]:
df_numerical = jd_2.copy()
encoder_dict = {}

for col in string_variables:
    le = preprocessing.LabelEncoder()
    le.fit(jd_2[col])
    encoder_dict[col] = le
    df_numerical[col] = le.transform(jd_2[col])

df_numerical.head()

,jobs,jobtitle,dictionary
0,129,event coordinator sales marketing account mana...,event coordinator sales marketing account mana...
1,143,account coordinator it,account coordinator it account coordinator it ...
2,129,account project coordinator,account project coordinator account project co...
3,3,account director advertising agency,account director advertising agency account di...
4,3,sr account director,sr account director sr account director classi...


In [8]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(jd_2['dictionary'], jd_2['jobs'], random_state=1)

In [9]:
#feat = list(df_numerical.dictionary.values.flatten())
#type(feat)
#feat = unicode(feat, 'utf-8')

In [10]:
#df_numerical.dictionary[0]
#import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')
y_test


array(['business intelligence analysts', 'cost estimators',
       'automotive master mechanics', ...,
       'automotive specialty technicians', 'cashiers',
       'stock clerks sales floor'], dtype=object)

In [11]:
X_test.shape

(1328L,)

In [12]:
X_train.shape

(3982L,)

In [13]:
type(X_train)

numpy.ndarray

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(decode_error = 'ignore')
#vect = CountVectorizer()
#vect.fit(X_train)
#vect.transform(X_train)

In [15]:
#vect.get_feature_names()
# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train)
train_dtm

<3982x5484 sparse matrix of type '<type 'numpy.int64'>'
	with 41133 stored elements in Compressed Sparse Row format>

In [16]:
# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test)
test_dtm

<1328x5484 sparse matrix of type '<type 'numpy.int64'>'
	with 12787 stored elements in Compressed Sparse Row format>

In [17]:
train_features = vect.get_feature_names()
len(train_features)

5484

In [18]:
#train_features[:50]

In [22]:
#train_features[-50:]

In [19]:
# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
# exercise: create a DataFrame of tokens with their counts.
train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_arr, axis=0)})
train_token_counts.sort_index(by='count', ascending=False)

,count,token
2941,1317,manager
4476,1110,senior
380,896,assistant
4917,854,technician
1438,827,director
1675,761,engineer
4354,648,sales
386,604,associate
4492,559,service
1831,496,experience


In [29]:
# use SVM
from sklearn import svm
svc = svm.SVC(kernel='linear', probability = True)

svc.fit(train_dtm, y_train)
#Predict 


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [30]:
# make predictions on test data using test_dtm
preds = svc.predict(test_dtm)
preds

array(['business intelligence analysts', 'cost estimators',
       'bus and truck mechanics and diesel engine specialists', ...,
       'billing cost and rate clerks', 'cashiers',
       'stock clerks sales floor'], dtype=object)

In [31]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.610692771084
[[5 0 0 ..., 0 0 0]
 [0 7 0 ..., 0 0 0]
 [0 0 3 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 6 0 0]
 [0 0 0 ..., 0 9 0]
 [0 0 0 ..., 0 0 6]]


In [32]:
# predict (poorly calibrated) probabilities and calculate AUC
probs = svc.predict_proba(test_dtm)[:, 1]
probs
#print(metrics.roc_auc_score(y_test, probs))

array([ 0.00044693,  0.00158553,  0.00042845, ...,  0.0041326 ,
        0.00496447,  0.00134019])

In [29]:
# exercise: show the message text for the false positives
#X_test[y_test < preds]

In [30]:

# exercise: show the message text for the false negatives
#X_test[y_test > preds]

In [33]:
from sklearn.metrics import classification_report 

print(classification_report(y_test, preds) )

             precision    recall  f1-score   support

accountants       0.56      0.83      0.67         6
administrative assistants       0.58      0.64      0.61        11
administrative services managers       0.43      0.60      0.50         5
advertising and promotions managers       0.62      0.80      0.70        10
advertising sales agents       0.67      0.50      0.57         8
art directors       0.45      1.00      0.62         5
assemblers and fabricators       0.21      0.40      0.28        10
audio and video equipment technicians       1.00      0.40      0.57         5
   auditors       0.83      0.83      0.83         6
automotive master mechanics       0.33      0.12      0.18         8
automotive specialty technicians       0.75      0.38      0.50         8
 bartenders       0.82      1.00      0.90         9
bill and account collectors       0.50      0.50      0.50         4
billing cost and rate clerks       0.42      0.71      0.53         7
biological technici

In [34]:

target_predicted = svc.predict(test_dtm)
target_prob = svc.predict_proba(test_dtm)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(y_test, target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,accountants,administrative assistants,administrative services managers,advertising and promotions managers,advertising sales agents,art directors,assemblers and fabricators,audio and video equipment technicians,auditors,automotive master mechanics,...,travel agents,treasurers and controllers,truck drivers heavy and tractor trailer,truck drivers light or delivery services,vocational education teachers postsecondary,waiters and waitresses,web developers,welders cutters and welder fitters,wholesale and retail buyers except farm products,All
True,,,,,,,,,,,,,,,,,,,,,
accountants,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
administrative assistants,0,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
administrative services managers,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
advertising and promotions managers,0,0,0,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
advertising sales agents,0,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
art directors,0,0,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
assemblers and fabricators,0,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,10
audio and video equipment technicians,0,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,1,5
auditors,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,6


In [36]:
from sklearn.cross_validation import cross_val_score
cross_val_score(svc, train_dtm, y_train, cv=5, scoring='accuracy').mean()

0.48037582698528575

In [34]:
# try using all of the data

In [38]:
train_dtm = vect.fit_transform(jd_2['dictionary'])


In [39]:
# train a SVC model using train_dtm
svc = svm.SVC(kernel='linear', probability = True)


svc.fit(train_dtm, jd_2['jobs'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [40]:
preds = svc.predict(train_dtm)
preds.shape

(5310L,)

In [41]:

print(classification_report(jd_2['jobs'], preds) )

             precision    recall  f1-score   support

accountants       0.86      1.00      0.92        30
administrative assistants       0.83      1.00      0.91        30
administrative services managers       0.64      0.97      0.77        30
advertising and promotions managers       0.88      0.97      0.92        30
advertising sales agents       1.00      0.60      0.75        30
art directors       1.00      0.90      0.95        30
assemblers and fabricators       0.59      0.97      0.73        30
audio and video equipment technicians       0.97      0.93      0.95        30
   auditors       0.93      0.90      0.92        30
automotive master mechanics       0.93      0.90      0.92        30
automotive specialty technicians       0.94      0.97      0.95        30
 bartenders       1.00      0.93      0.97        30
bill and account collectors       0.97      0.93      0.95        30
billing cost and rate clerks       0.91      0.97      0.94        30
biological technici

In [42]:

target_predicted = svc.predict(train_dtm)
target_prob = svc.predict_proba(train_dtm)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(jd_2['jobs'], target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,accountants,administrative assistants,administrative services managers,advertising and promotions managers,advertising sales agents,art directors,assemblers and fabricators,audio and video equipment technicians,auditors,automotive master mechanics,...,travel agents,treasurers and controllers,truck drivers heavy and tractor trailer,truck drivers light or delivery services,vocational education teachers postsecondary,waiters and waitresses,web developers,welders cutters and welder fitters,wholesale and retail buyers except farm products,All
True,,,,,,,,,,,,,,,,,,,,,
accountants,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30
administrative assistants,0,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30
administrative services managers,0,0,29,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30
advertising and promotions managers,0,0,0,29,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30
advertising sales agents,0,0,0,0,18,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30
art directors,0,0,0,2,0,27,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30
assemblers and fabricators,0,0,0,0,0,0,29,0,0,0,...,0,0,0,0,0,0,0,0,0,30
audio and video equipment technicians,0,0,0,0,0,0,0,28,0,0,...,0,0,0,0,0,0,0,0,0,30
auditors,0,0,0,0,0,0,0,0,27,0,...,0,0,0,0,0,0,0,0,0,30


In [44]:
from sklearn.cross_validation import cross_val_score
cross_val_score(svc, train_dtm, jd_2['jobs'], cv=5, scoring='accuracy').mean()

0.46553672316384176

In [46]:
from sklearn.pipeline import Pipeline

from sklearn.grid_search import GridSearchCV

In [50]:
#Check to see what kernel we should use for the SVM
pipeline = Pipeline([('svm', svm.SVC(C=1.0, kernel='linear'))])
param_grid = {
    
    'svm__kernel': ['rbf', 'linear', 'poly','sigmoid']
}
clf = GridSearchCV(pipeline, param_grid)
clf = clf.fit(train_dtm, jd_2['jobs'])
clf.best_params_

{'svm__kernel': 'linear'}